# Análisis de estadísticas de equipos de la MLB
En este notebook, analizaremos estadísticas básicas y avanzadas de equipos de la MLB para los últimos 5 años.

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

## Función para obtener datos de equipos
La siguiente función recupera estadísticas básicas de equipos de la MLB para un año específico usando la base de datos de Lahman.

In [12]:
# Función para cargar y filtrar datos
def load_and_filter_data(years):
    """
    Carga el archivo CSV de Lahman y selecciona las columnas relevantes.
    """
    teams_path = "../data/raw/Teams.csv"
    
    try:
        df = pd.read_csv(teams_path)
        # Filtramos por años
        df = df[df['yearID'].isin(years)]
        # Seleccionamos las columnas relevantes
        # cols = ['yearID', 'teamID', 'W', 'L', 'R', 'AB', 'H', 'BB', 'SO', 'HR', 'ER'] # basicas
        cols = ['yearID', 'teamID', 'W', 'L', 'R', 'AB', 'H', 'BB', 'SO',
               'HR', 'ER', '2B', '3B', 'HBP', 'SF', 'IPouts', 'HA', 'BBA',
               'SOA', 'E']
        return df[cols]
    except Exception as e:
        print(f"Error cargando los datos: {e}")
        return None

## Función para calcular métricas avanzadas
Calculamos métricas avanzadas como OBP, SLG, OPS y porcentaje de bases por bolas.

In [13]:
def calculate_metrics(df):
    """
    Calcula métricas de béisbol basadas en estadísticas básicas
    """
    # Calculamos OBP (On Base Percentage)
    df['OBP'] = (df['H'] + df['BB']) / (df['AB'] + df['BB'])
    
    # Calculamos SLG (Slugging Percentage)
    df['SLG'] = (df['H'] + df['HR'] * 3) / df['AB']
    
    # Calculamos OPS
    df['OPS'] = df['OBP'] + df['SLG']
    
    # Calculamos BB% (Base on Balls Percentage)
    df['BB%'] = df['BB'] / (df['AB'] + df['BB']) * 100
    
    return df

def calculate_advanced_metrics(df):
    """
    Calcula métricas avanzadas de béisbol basadas en estadísticas básicas
    """
    # Métricas Ofensivas
    # ------------------
    
    # Singles (1B) = Hits - (2B + 3B + HR)
    df['1B'] = df['H'] - (df['2B'] + df['3B'] + df['HR'])
    
    # Total Bases (TB)
    df['TB'] = df['1B'] + (2 * df['2B']) + (3 * df['3B']) + (4 * df['HR'])
    
    # Batting Average (AVG)
    df['AVG'] = df['H'] / df['AB']
    
    # On-Base Percentage (OBP)
    df['PA'] = df['AB'] + df['BB'] + df['HBP'] + df['SF']  # Plate Appearances
    df['OBP'] = (df['H'] + df['BB'] + df['HBP']) / df['PA']
    
    # Slugging Percentage (SLG)
    df['SLG'] = df['TB'] / df['AB']
    
    # On-base Plus Slugging (OPS)
    df['OPS'] = df['OBP'] + df['SLG']
    
    # Base on Balls Percentage (BB%)
    df['BB%'] = df['BB'] / df['PA'] * 100
    
    # Strike Out Percentage (K%)
    df['K%'] = df['SO'] / df['PA'] * 100
    
    # ISO (Isolated Power)
    df['ISO'] = df['SLG'] - df['AVG']
    
    # Métricas Defensivas y Pitcheo
    # -----------------------------
    
    # Innings Pitched (IP)
    df['IP'] = df['IPouts'] / 3
    
    # Earned Run Average (ERA)
    df['ERA'] = (df['ER'] * 9) / df['IP']
    
    # WHIP (Walks and Hits per Inning Pitched)
    df['WHIP'] = (df['HA'] + df['BBA']) / df['IP']
    
    # K/9 (Strikeouts per 9 innings)
    df['K/9'] = (df['SOA'] * 9) / df['IP']
    
    # BB/9 (Walks per 9 innings)
    df['BB/9'] = (df['BBA'] * 9) / df['IP']
    
    # K/BB Ratio
    df['K/BB'] = df['SOA'] / df['BBA']
    
    # Métricas de Equipo
    # -----------------
    
    # Winning Percentage
    df['WIN%'] = df['W'] / (df['W'] + df['L'])
    
    # Runs per Game
    df['R/G'] = df['R'] / (df['W'] + df['L'])
    
    # Defensive Efficiency Ratio (DER)
    # (1 - ((H - HR) + E) / (AB - K - HR + SF))
    df['DER'] = 1 - ((df['HA'] - df['HR'] + df['E']) / 
                     (df['AB'] - df['SO'] - df['HR'] + df['SF']))
    
    return df

## Recolectar y procesar datos
Analizamos los datos de los últimos 5 años, calculamos métricas avanzadas y combinamos los resultados.

## Mostrar resultados
Revisamos una muestra de los datos procesados, estadísticas descriptivas y valores faltantes.

In [14]:
# Años para los cuales queremos procesar los datos
years = range(2019, 2024)

# Cargar y procesar datos
data = load_and_filter_data(years)
if data is not None:
    # data = calculate_metrics(data)
    data = calculate_advanced_metrics(data)
    # Guardar el resultado
    data.to_csv("../data/processed/team_stats.csv", index=False)
    print("Datos procesados y guardados exitosamente.")

    # Mostrar muestra de los datos procesados
    print("Muestra de los datos:")
    display(data.head())

    # Estadísticas básicas
    print("\nEstadísticas descriptivas:")
    display(data.describe())
    
    # Verificar si existen valores faltantes
    print("\nValores faltantes por columna:")
    display(data.isnull().sum())
else:
    print("No se pudieron procesar los datos.")

Datos procesados y guardados exitosamente.
Muestra de los datos:



Valores faltantes por columna:


yearID    0
teamID    0
W         0
L         0
R         0
AB        0
H         0
BB        0
SO        0
HR        0
ER        0
2B        0
3B        0
HBP       0
SF        0
IPouts    0
HA        0
BBA       0
SOA       0
E         0
1B        0
TB        0
AVG       0
PA        0
OBP       0
SLG       0
OPS       0
BB%       0
K%        0
ISO       0
IP        0
ERA       0
WHIP      0
K/9       0
BB/9      0
K/BB      0
WIN%      0
R/G       0
DER       0
dtype: int64